In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import re
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from csv_parser import CSVParser
from markovSquares import markovSquares, markov_index_inner

In [3]:
# Define categories based on ranges or discrete values
total_df = pd.read_csv("../../Project materials(1)/resampled_data_20min.csv")

# total_df = pd.read_csv("../../resources/resampled_data_h.csv")

total_df['etaParsed'] = pd.to_datetime(total_df['etaParsed'])
total_df["time"] = pd.to_datetime(total_df['time'])

start_date = pd.to_datetime('2024-01-01')

total_df["etaParsed"] = (total_df['etaParsed'] - start_date).dt.days


total_df = total_df.drop(columns=["navstat"])



time_diffs = total_df["time"].diff()
time_interval = time_diffs.dropna().iloc[0]
time_interval = int(time_interval.total_seconds()/(60*20))



total_df.set_index("time", inplace=True)


In [4]:
print(total_df.head(20))

                       cog   sog  rot  heading  latitude  longitude  \
time                                                                  
2024-01-01 00:00:00  284.0   0.7  0.0     88.0 -34.74370  -57.85130   
2024-01-01 00:20:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 00:40:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 01:00:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 01:20:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 01:40:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 02:00:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 02:20:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 02:40:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 03:00:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 03:20:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-01-01 03:40:00   92.8  14.2  0.0     90.0 -35.16787  -56.77210   
2024-0

In [5]:
sub_df = total_df.head(20)
transition_matrix = markovSquares(sub_df, "vesselId", 2)
transition_matrix[72, 151, 4]

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


/home/susan-palencia/Documents/Maskinlæring/OceanWarlock/src/markovSquares.py:35: RuntimeWarning: invalid value encountered in divide
  normalized_markov_matrix = markov_matrix / np.sum(markov_matrix, axis=-1, keepdims=True)


0.8947368421052632